### Self-study Colab Activity 6.2: Interpreting the Results of K-Means and PCA




In this activity, you are tasked with profiling customer groups for a large telecommunications company.  The data provided contains information on customers' purchasing and usage behavior with telecom products.  Your goal is to use PCA and clustering to segment these customers into meaningful groups, and report back your findings.  

Because these results need to be interpretable, it is important to keep the number of clusters reasonable.  Think about how you might represent some of the non-numeric features so that they can be included in your segmentation models.  You are to report back your approach and findings to the class.  Be specific about what features were used and how you interpret the resulting clusters.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN

In [2]:
df = pd.read_csv('data/telco_churn_data.csv')

In [3]:
df.head()

,Customer ID,Referred a Friend,Number of Referrals,Tenure in Months,Offer,Phone Service,Avg Monthly Long Distance Charges,Multiple Lines,Internet Service,Internet Type,...,Latitude,Longitude,Population,Churn Value,CLTV,Churn Category,Churn Reason,Total Customer Svc Requests,Product/Service Issues Reported,Customer Satisfaction
0,8779-QRDMV,No,0,1,NaN,No,0.00,No,Yes,Fiber Optic,...,34.023810,-118.156582,68701,1,5433,Competitor,Competitor offered more data,5,0,NaN
1,7495-OOKFY,Yes,1,8,Offer E,Yes,48.85,Yes,Yes,Cable,...,34.044271,-118.185237,55668,1,5302,Competitor,Competitor made better offer,5,0,NaN
2,1658-BYGOY,No,0,18,Offer D,Yes,11.33,Yes,Yes,Fiber Optic,...,34.108833,-118.229715,47534,1,3179,Competitor,Competitor made better offer,1,0,NaN
3,4598-XLKNJ,Yes,1,25,Offer C,Yes,19.76,No,Yes,Fiber Optic,...,33.936291,-118.332639,27778,1,5337,Dissatisfaction,Limited range of services,1,1,2.0
4,4846-WHAFZ,Yes,1,37,Offer C,Yes,6.33,Yes,Yes,Cable,...,33.972119,-118.020188,26265,1,2793,Price,Extra data charges,1,0,2.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 46 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Customer ID                        7043 non-null   object 
 1   Referred a Friend                  7043 non-null   object 
 2   Number of Referrals                7043 non-null   int64  
 3   Tenure in Months                   7043 non-null   int64  
 4   Offer                              3166 non-null   object 
 5   Phone Service                      7043 non-null   object 
 6   Avg Monthly Long Distance Charges  7043 non-null   float64
 7   Multiple Lines                     7043 non-null   object 
 8   Internet Service                   7043 non-null   object 
 9   Internet Type                      5517 non-null   object 
 10  Avg Monthly GB Download            7043 non-null   int64  
 11  Online Security                    7043 non-null   objec

In [5]:
df.describe()

,Number of Referrals,Tenure in Months,Avg Monthly Long Distance Charges,Avg Monthly GB Download,Monthly Charge,Total Regular Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Age,Number of Dependents,Zip Code,Latitude,Longitude,Population,Churn Value,CLTV,Total Customer Svc Requests,Product/Service Issues Reported,Customer Satisfaction
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,1834.000000
mean,1.951867,32.386767,22.958954,21.110890,65.538800,2280.381264,1.962182,278.499225,749.099262,46.509726,0.468692,93486.070567,36.197455,-119.756684,22139.603294,0.265370,4400.295755,1.338776,0.308107,3.005453
std,3.001199,24.542061,15.448113,20.948471,30.606805,2266.220462,7.902614,685.039625,846.660055,16.750352,0.962802,1856.767505,2.468929,2.154425,21152.392837,0.441561,1183.057152,1.430471,0.717514,1.256938
min,0.000000,1.000000,0.000000,0.000000,18.250000,18.800000,0.000000,0.000000,0.000000,19.000000,0.000000,90001.000000,32.555828,-124.301372,11.000000,0.000000,2003.000000,0.000000,0.000000,1.000000
25%,0.000000,9.000000,9.210000,3.000000,35.890000,400.150000,0.000000,0.000000,70.545000,32.000000,0.000000,92101.000000,33.990646,-121.788090,2344.000000,0.000000,3469.000000,0.000000,0.000000,2.000000
50%,0.000000,29.000000,22.890000,17.000000,71.968000,1394.550000,0.000000,0.000000,401.440000,46.000000,0.000000,93518.000000,36.205465,-119.595293,17554.000000,0.000000,4527.000000,1.000000,0.000000,3.000000
75%,3.000000,55.000000,36.395000,28.000000,90.650000,3786.600000,0.000000,182.620000,1191.100000,60.000000,0.000000,95329.000000,38.161321,-117.969795,36125.000000,1.000000,5380.500000,2.000000,0.000000,4.000000
max,11.000000,72.000000,49.990000,94.000000,123.084000,8684.800000,49.790000,6477.000000,3564.720000,80.000000,9.000000,96150.000000,41.962127,-114.192901,105285.000000,1.000000,6500.000000,9.000000,6.000000,5.000000


In [6]:
object_list = df.select_dtypes('object').columns.tolist()
churn_numeric = df.drop(object_list, axis= 1)
churn_numeric.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Number of Referrals                7043 non-null   int64  
 1   Tenure in Months                   7043 non-null   int64  
 2   Avg Monthly Long Distance Charges  7043 non-null   float64
 3   Avg Monthly GB Download            7043 non-null   int64  
 4   Monthly Charge                     7043 non-null   float64
 5   Total Regular Charges              7043 non-null   float64
 6   Total Refunds                      7043 non-null   float64
 7   Total Extra Data Charges           7043 non-null   float64
 8   Total Long Distance Charges        7043 non-null   float64
 9   Age                                7043 non-null   int64  
 10  Number of Dependents               7043 non-null   int64  
 11  Zip Code                           7043 non-null   int64

In [7]:
churn_numeric_nona = churn_numeric.dropna()

In [8]:
drop_list = ['Zip Code','Latitude','Longitude', 'Population']
churn_clean = churn_numeric_nona.drop(drop_list, axis = 1)

In [9]:
churn_clean['Gender'] = df['Gender'].map({'Female':1, 'Male':2})
churn_clean['Married'] = df['Married'].map({'No':1, 'Yes':2})
churn_clean['Contract'] = df['Contract'].map({'Month-to-Month':1, 'One Year':2, 'Two Year':3})
churn_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1834 entries, 3 to 7040
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Number of Referrals                1834 non-null   int64  
 1   Tenure in Months                   1834 non-null   int64  
 2   Avg Monthly Long Distance Charges  1834 non-null   float64
 3   Avg Monthly GB Download            1834 non-null   int64  
 4   Monthly Charge                     1834 non-null   float64
 5   Total Regular Charges              1834 non-null   float64
 6   Total Refunds                      1834 non-null   float64
 7   Total Extra Data Charges           1834 non-null   float64
 8   Total Long Distance Charges        1834 non-null   float64
 9   Age                                1834 non-null   int64  
 10  Number of Dependents               1834 non-null   int64  
 11  Churn Value                        1834 non-null   int64  
 1

In [13]:
from sklearn.decomposition import PCA
from scipy.linalg import svd
import plotly.express as px
import plotly.graph_objects as go

churn_scaled = (churn_clean - churn_clean.mean())/churn_clean.std()
U, sigma, VT = svd(churn_scaled)
percent_variance_explained = sigma/sigma.sum()
cumulative_variance_ratio = np.cumsum(sigma)/sigma.sum()

#identify 85% threshold

number_of_components_for_85 = int((np.cumsum(percent_variance_explained)< .85).sum())
print("The number of components needed is: ",number_of_components_for_85)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x = np.arange(1, len(cumulative_variance_ratio)+1),
    y = cumulative_variance_ratio,
    mode = 'lines+markers',
    line = dict(dash = 'dash'),
    name = "Cumlative Variance"
))
fig.add_shape(type = 'line', x0 =1, x1 = len(cumulative_variance_ratio),
              y0 = 0.85, y1 = 0.85, line = dict(color = 'red'))
fig.add_annotation(x = 1, y =0.8, text = '.85% Threshold', showarrow=False)
fig.add_shape(type = 'line',
              x0 = number_of_components_for_85,
              x1 = number_of_components_for_85,
              y0 = 0, y1 = 1,
              line = dict(color = 'red', dash = 'dash'))
fig.update_layout(
    title = 'Variance By Components',
    xaxis_title = 'Number of Components',
    yaxis_title = 'Percent Variance'
)
fig.show()


pca = PCA(n_components = number_of_components_for_85, random_state = 42)
components = pca.fit_transform(churn_scaled)
components.shape


The number of components needed is:  13


(1834, 13)

In [21]:

inertias = []
for i in range(1,14):
  kmeans = KMeans(n_clusters=i, n_init= 'auto').fit(components)
  inertias.append(kmeans.inertia_)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x = np.arange(1, len(cumulative_variance_ratio)+1),
    y = inertias,
    mode = 'lines+markers'
))
fig.update_layout(
    xaxis_title = 'Number of Clusters *k*',
    yaxis_title = 'Inertias',
    title = "Elbow Method - Inertias by Number of Clusters"
)
fig.show()

kmeans = KMeans(n_clusters = 4, n_init = 'auto').fit(components)
churn_clean['cluster'] = kmeans.labels_
churn_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1834 entries, 3 to 7040
Data columns (total 20 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Number of Referrals                1834 non-null   int64  
 1   Tenure in Months                   1834 non-null   int64  
 2   Avg Monthly Long Distance Charges  1834 non-null   float64
 3   Avg Monthly GB Download            1834 non-null   int64  
 4   Monthly Charge                     1834 non-null   float64
 5   Total Regular Charges              1834 non-null   float64
 6   Total Refunds                      1834 non-null   float64
 7   Total Extra Data Charges           1834 non-null   float64
 8   Total Long Distance Charges        1834 non-null   float64
 9   Age                                1834 non-null   int64  
 10  Number of Dependents               1834 non-null   int64  
 11  Churn Value                        1834 non-null   int64  
 1

In [24]:
fig = px.scatter(x = components[:, 0], y = components[:, 1], color =kmeans.labels_, title = 'Clusters of Customers')
fig.update_xaxes(title = 'Principal Component 1')
fig.update_yaxes(title = 'Principal Component 2')
fig.show()

loadings = pca.components_.T*np.sqrt(pca.explained_variance_)
loading_matrix = pd.DataFrame(loadings, columns = [f'PC{i+1}' for i in range(loadings.shape[1])], index = churn_clean.columns)
loading_matrix

sorted_indices = {f'PC{i+1}':loading_matrix[f'PC{i+1}'].abs().sort_values(ascending=False).index
                  for i in range(loading_matrix.shape[1])}

loading_matrix_sorted_all = pd.DataFrame(sorted_indices)
loading_matrix_sorted_all = loading_matrix_sorted_all[:3]
loading_matrix_sorted_all




,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13
0,Tenure in Months,Monthly Charge,Avg Monthly GB Download,Married,Avg Monthly Long Distance Charges,Gender,Total Refunds,CLTV,Product/Service Issues Reported,CLTV,Number of Dependents,Total Customer Svc Requests,Total Extra Data Charges
1,Churn Value,cluster,Age,Number of Referrals,Total Long Distance Charges,Total Refunds,Gender,Monthly Charge,Total Customer Svc Requests,Contract,Age,Number of Dependents,Total Customer Svc Requests
2,Contract,Total Regular Charges,Total Extra Data Charges,Number of Dependents,Number of Dependents,CLTV,CLTV,Age,Total Refunds,Monthly Charge,Number of Referrals,Product/Service Issues Reported,Monthly Charge
